## Group 14C - Fake News Detector Project

---




---


AI4ALL Ignite - Fall 2024 : Wynne Conger, Nina Elmoyan, Ramneek Kaur, Rhode Sanchez


In [ ]:
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')